In [ ]:
import pandas as pd
import numpy as np
import os
import re

# Create Dataset

## Install libraries for convert pdf to text

In [ ]:
!pip install PyPDF2
import PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 220 kB 5.2 MB/s 


## Read Files

In [ ]:
ROOT_DIRECTORY = './Job Recommender Dataset'

## Create dataset

In [ ]:
def create_dataset(root_directory):
  row = []
  for i in range(len([x[0] for x in os.walk(root_directory)])):
    filelist = os.listdir([x[0] for x in os.walk(root_directory)][i])
    for f in filelist:
      if f.endswith(".pdf"):
        pdfFileObj = open([x[0] for x in os.walk(root_directory)][i] + '/' + f, 'rb')
        try:
          pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        except:
          continue
        number_of_pages = pdfReader.numPages
        content = []
        for page in range(number_of_pages):
          pageObj = pdfReader.getPage(page)
          content.append(
              {
                  "page": page,
                  "content": pageObj.extractText()
              }
          )

        row.append(
            {
                "id": str(f).split(".")[0],
                "category": str([x[0] for x in os.walk(root_directory)][i]).split('/')[-1],
                "text": content
            }
        )
  pdfFileObj.close()
  return row


In [ ]:
dataset = pd.DataFrame(create_dataset(ROOT_DIRECTORY))

In [ ]:
dataset

,id,category,text
0,cc5dfda4-3e9b-47a6-9b96-d36bbce44a6d (1),Product Designer,"[{'page': 0, 'content': 'Educa tion course LAN..."
1,cc5dfda4-3e9b-47a6-9b96-d36bbce44a6d,Product Designer,"[{'page': 0, 'content': 'Educa tion course LAN..."
2,def4d72c-0056-440a-b08a-8f033571ad7e (1),Product Designer,"[{'page': 0, 'content': '` ..."
3,def4d72c-0056-440a-b08a-8f033571ad7e,Product Designer,"[{'page': 0, 'content': '` ..."
4,6a6b5316-bb02-4346-9bef-a99dadb21b56,Product Designer,"[{'page': 0, 'content': 'Exper t Mid LowS k i ..."
...,...,...,...
479,e36f8abf-8f1e-48c4-981e-64c9357c0777,Finance & Accounting Supervisor,"[{'page': 0, 'content': 'مشخصات فردی ..."
480,93247d28-5fbd-428d-973c-7411bd42742a,Finance & Accounting Supervisor,"[{'page': 0, 'content': 'ﯽﺼﺨﺷ ﯽﺼﺨﺷ تﺎﻋﻼﻃا ..."
481,f2d8d625-b88a-4dd0-87c3-a2d6ebf089fd,Finance & Accounting Supervisor,"[{'page': 0, 'content': 'ﯽﺼﺨﺷ ﯽﺼﺨﺷ تﺎﻋﻼﻃا ..."
482,03ab5376-5083-4bb2-a13a-39aff49c6216,Finance & Accounting Supervisor,"[{'page': 0, 'content': ' موضوع :رزومه كاري..."


# Data preprocessing

In [ ]:
def clean_text(text):
  text = text.lower()
  text = re.sub('\n', '.', text)
  text = re.sub('[^.آ-یa-z]+', ' ', text)
  text = re.sub('\d+', 'N', text)
  return text

## Detect language of each resume

In [ ]:
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 5.0 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=7cd8066f38a515cdd2a7ad7d21af7d079e01a323d4fb31712c048d95a2dbf41e
  Stored in directory: /root/.cache/pip/wheels/c5/96/8a/f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect


In [ ]:
from langdetect import detect

lang = []
for i in range(len(dataset)):
  try:
    lang.append(
        detect(dataset['text'].iloc[i][0]['content'])
    )
  except:
    lang.append(
        None
    )

dataset['language'] = lang

In [ ]:
dataset['text'].iloc[0]

"[{'page': 0, 'content': 'Educa tion\\ncourse\\nLANGUAGEexperience\\r\\nUniversity of Science and Cultur e\\nPh.D. Management || 2014-Current\\n\\nUniversity of Sheikh Bahaei\\nMSc.T ourism Management  || 201 1-2013\\n\\nUniversity of Payamnoor   \\nBA. T ourism Management || 2007-201 1 \\nFr ontend developer\\nCodingfrond ||2022\\r\\n\\nPr oduct design\\nDexign Academy ||2021\\r\\n\\nDigital photography\\nInstitute of T echnology || 2020\\nW ritingReadingEnglish\\nListening\\nspeaking\\nW ritingReadingAzerbaijani \\nListening\\nspeakingFatemeh Farahani\\nHobbies\\nt raveling\\nListen t o music\\nreading\\nphotographPr oduct design\\nFarahanif1988@gmail.com\\nMoallem St, Shariati St, T ehran, Iran  Fatemeh.Farahani\\nU s e r  I n t e r f a c e  d e s i g n\\rA d o b e  P h o t o s h o p C r i t i c a l  T h i n k i n g\\r\\nFigma\\nZ eplin\\nj avascript\\nH t m l / C S S\\nM i c r o s o f t  O f f i c e  Pr oduct design\\n Cryptoeye, T ehran, j an 2022- Current\\n\\n U X  R esear cher 

# Export a plain text out of whole CV content

In [ ]:
resume_content = []
for i in range(len(dataset)):
  page_content = []
  for j in range(len(dataset['text'].iloc[i])):
    page_content.append(
        clean_text(
            dataset['text'].iloc[i][j]['content']
        )
    )
  resume_content.append(
      ".".join(page_content)
  )

dataset['resume_content'] = resume_content

In [ ]:
dataset['resume_content'].iloc[0]

'educa tion.course.languageexperience .university of science and cultur e.ph.d. management current..university of sheikh bahaei.msc.t ourism management ..university of payamnoor .ba. t ourism management .fr ontend developer.codingfrond ..pr oduct design.dexign academy ..digital photography.institute of t echnology .w ritingreadingenglish.listening.speaking.w ritingreadingazerbaijani .listening.speakingfatemeh farahani.hobbies.t raveling.listen t o music.reading.photographpr oduct design.farahanif gmail.com.moallem st shariati st t ehran iran fatemeh.farahani.u s e r i n t e r f a c e d e s i g n a d o b e p h o t o s h o p c r i t i c a l t h i n k i n g .figma.z eplin.j avascript.h t m l c s s.m i c r o s o f t o f f i c e pr oduct design. cryptoeye t ehran j an current.. u x r esear cher part t ime . t asty t ehran j ul current ..user e x perience c onsulting part t ime .bourseye t ehran may j an ..u x r esear cher.o tagha k t ehran a pr may ..content m anager u x r esear cher.lastse

In [ ]:
#########################################################
dataset.to_csv("job_recommender_dataset.csv", index=False)

# Read frequent english words (including words, technical skills & Iranian universities)

In [ ]:
with open('./english_frequent_words.txt') as f:
  en_words = f.readlines()

english_unigrams = [word.strip() for word in en_words]

with open('./persian_frequent_words.txt') as f:
  fa_words = f.readlines()

persian_unigrams = [word.strip() for word in fa_words]

## Word Segmentation Spell Checker

In [ ]:
def word_segmentation_spell_checker(string):
  tokens = []
  # lowercaseCorpus = [x.lower() for x in english_unigrams]
  lowercaseCorpus = [x.lower() for x in persian_unigrams]
  i = 0
  while i < len(string):
      maxWord = ""
      for j in range(i, len(string)):
          tempWord = string[i:j+1]
          if tempWord in lowercaseCorpus and len(tempWord) > len(maxWord):
              maxWord = tempWord
      i = i+len(maxWord)
      tokens.append(maxWord)

  return tokens

In [ ]:
example = 'languageexperience'
print(f"word segmentation: {word_segmentation_spell_checker(example)}")

word segmentation: ['language', 'experience']


## Remove Extra White Spaces and then, Spell Checking

In [ ]:
def fix_extra_white_spaces(string):
  merged_text = re.sub("\s+","", string)
  res = " ".join(word_segmentation_spell_checker(merged_text))
  if res == "":
    pass
  else:
    return res

In [ ]:
example = "u x r esear cher"
print(f'fix example: {fix_extra_white_spaces(example)}')

fix example: ux researcher


## Set Interruption Signal

In [ ]:
import signal
from contextlib import contextmanager
class TimeoutException(Exception):
  pass

@contextmanager
def time_limit(seconds):
  def signal_handler(signum, frame):
    raise TimeoutException("Timed out!")
  signal.signal(signal.SIGALRM, signal_handler)
  signal.alarm(seconds)
  try:
    yield
  finally:
    signal.alarm(0)


## Clean the whole resume_content and create refined_content

In [ ]:
contents = []
for i in range(len(dataset)):
  if (dataset['language'].iloc[i] == 'en') or (dataset['language'].iloc[i] == 'fa'):
    refined_content = []
    for j in range(len(dataset['resume_content'].iloc[i].split("."))):
      try:
        with time_limit(5):
          refined_content.append(
              fix_extra_white_spaces(dataset['resume_content'].iloc[i].split(".")[j])
          )
      except:
        pass
    refined_content = list(filter(lambda item: item is not None, refined_content))
    contents.append(
        ". ".join(refined_content)
    )
  else:
    contents.append(None)

In [ ]:
dataset['refined_content'] = contents
dataset

In [ ]:
dataset.to_csv("./cvs_dataset_with_clean_content.csv", index=False)

In [ ]:
###############################################################
dataset = pd.read_csv("./cvs_dataset_with_clean_content.csv")
dataset

,id,category,text,language,resume_content,refined_content
0,cc5dfda4-3e9b-47a6-9b96-d36bbce44a6d (1),Product Designer,"[{'page': 0, 'content': 'Educa tion\ncourse\nL...",en,educa tion.course.languageexperience .universi...,education. course. language experience. univer...
1,cc5dfda4-3e9b-47a6-9b96-d36bbce44a6d,Product Designer,"[{'page': 0, 'content': 'Educa tion\ncourse\nL...",en,educa tion.course.languageexperience .universi...,education. course. language experience. univer...
2,def4d72c-0056-440a-b08a-8f033571ad7e (1),Product Designer,"[{'page': 0, 'content': '` \n \n \n \n \n \n \...",en,. . . . . . . . . . . . . . . . . . . . . . . ...,linkedin. atabaki. portfolio. be hance. tab ak...
3,def4d72c-0056-440a-b08a-8f033571ad7e,Product Designer,"[{'page': 0, 'content': '` \n \n \n \n \n \n \...",en,. . . . . . . . . . . . . . . . . . . . . . . ...,linkedin. at aba ki. portfolio. be hance. tab ...
4,6a6b5316-bb02-4346-9bef-a99dadb21b56,Product Designer,"[{'page': 0, 'content': ""Exper t\nMid\nLowS k ...",en,exper t.mid.lows k i l l s.information ar chit...,expert. mid. lows kills. energetic ui. xd. hav...
...,...,...,...,...,...,...
479,e36f8abf-8f1e-48c4-981e-64c9357c0777,Finance & Accounting Supervisor,"[{'page': 0, 'content': 'مشخصات فردی ...",fa,مشخصات فردی .نام شیما نام خانوادگی حاجیان سال ...,NaN
480,93247d28-5fbd-428d-973c-7411bd42742a,Finance & Accounting Supervisor,"[{'page': 0, 'content': 'ﯽﺼﺨﺷ\nﯽﺼﺨﺷ\n \n \nتﺎﻋ...",fa,. . . .ت ا.ت ا. . . . . .ل . .نز. . . . . . . ...,NaN
481,f2d8d625-b88a-4dd0-87c3-a2d6ebf089fd,Finance & Accounting Supervisor,"[{'page': 0, 'content': 'ﯽﺼﺨﺷ\nﯽﺼﺨﺷ\n \n \nتﺎﻋ...",fa,. . . .ت ا.ت ا. . . . . .ل . .د . . . . . . .ن...,NaN
482,03ab5376-5083-4bb2-a13a-39aff49c6216,Finance & Accounting Supervisor,"[{'page': 0, 'content': ' \n \nموضوع :رزومه كا...",fa,. .موضوع رزومه كاري . . مشخصات فردی . .نام و ن...,NaN


In [ ]:
trainable_dataset = dataset[dataset['refined_content'].notnull()].reset_index(drop=True)
trainable_dataset = trainable_dataset.drop_duplicates(subset=['refined_content'])
trainable_dataset

,id,category,text,language,resume_content,refined_content
0,cc5dfda4-3e9b-47a6-9b96-d36bbce44a6d (1),Product Designer,"[{'page': 0, 'content': 'Educa tion\ncourse\nL...",en,educa tion.course.languageexperience .universi...,education. course. language experience. univer...
2,def4d72c-0056-440a-b08a-8f033571ad7e (1),Product Designer,"[{'page': 0, 'content': '` \n \n \n \n \n \n \...",en,. . . . . . . . . . . . . . . . . . . . . . . ...,linkedin. atabaki. portfolio. be hance. tab ak...
3,def4d72c-0056-440a-b08a-8f033571ad7e,Product Designer,"[{'page': 0, 'content': '` \n \n \n \n \n \n \...",en,. . . . . . . . . . . . . . . . . . . . . . . ...,linkedin. at aba ki. portfolio. be hance. tab ...
4,6a6b5316-bb02-4346-9bef-a99dadb21b56,Product Designer,"[{'page': 0, 'content': ""Exper t\nMid\nLowS k ...",en,exper t.mid.lows k i l l s.information ar chit...,expert. mid. lows kills. energetic ui. xd. hav...
5,0a820324-41f1-4bd5-bb29-5f059a813540 (2),Product Designer,"[{'page': 0, 'content': 'CONTACTUX/UI DESIGNER...",en,contactux ui designerf a r i m a.b a b a p o u...,baba pour. product features. would improve use...
...,...,...,...,...,...,...
198,56ffd402-152e-4d7c-9da4-74607d226e54,Content Marketing Specialist,"[{'page': 0, 'content': "" \nMohsen Heydarineja...",en,.mohsen heydarinejad .content specialist . . ....,content specialist. military service education...
199,52556678-60d0-4971-b568-d1d778081e10,Digital Marketing Specialist,"[{'page': 0, 'content': 'Social Media marketin...",en,social media marketing.mobina godarzi.digital ...,social media marketing. digital marketing. cur...
200,093259c9-4b98-4eb8-b1d3-295c5228bddd,Digital Marketing Specialist,"[{'page': 0, 'content': 'SUMMARY\nI am a Marke...",en,summary.i am a marketing manager specialist wh...,summary. years experience. ability. honesty is...
201,17f27cb0-df19-4f1c-9573-da306587efaa,Digital Marketing Specialist,"[{'page': 0, 'content': 'Iran, Tehran32 yrs ol...",en,iran tehran yrs old. . shahram shakibi b shahr...,shah rams ha ki bibs hah ram. sha ki big mail....


# Create Stopword list using TF-TDF scores


In [ ]:

all_cvs = '|'.join(trainable_dataset['refined_content'].tolist())
all_cvs = re.sub("[.]", "", all_cvs)
all_cvs = re.sub("\s+", " ", all_cvs)

from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
x = v.fit_transform(all_cvs.split("|"))

In [ ]:
tf_idf_df = pd.DataFrame(x.toarray(), columns = v.get_feature_names())
tf_idf_df

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aa,ab,aba,abetter,abi,abilities,ability,able,abo,about,...,zane,zas,zeina,zeplin,zer,zero,zig,zing,zo,zoom
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.097458,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.060862,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.071067,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.059383,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
186,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.078112,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.133314,0.0,0.0,0.0
187,0.0,0.0,0.000000,0.0,0.0,0.0,0.052218,0.0,0.0,0.033377,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
188,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0


## Detect tokens with the lowest TF-IDF score

In [ ]:
tf_idf_dict = {}
for i in list(tf_idf_df.columns):
  tf_idf_dict[str(i)] = tf_idf_df[str(i)].sum()

In [ ]:
{k: v for k, v in sorted(tf_idf_dict.items(), key=lambda item: item[1])}

{'afshin': 0.028646348317887208,
 'bills': 0.028646348317887208,
 'four': 0.028646348317887208,
 'getting': 0.028646348317887208,
 'iphone': 0.028646348317887208,
 'karim': 0.028646348317887208,
 'nine': 0.028646348317887208,
 'nonacademic': 0.028646348317887208,
 'plus': 0.028646348317887208,
 'sharif': 0.028646348317887208,
 'theres': 0.028646348317887208,
 'typesetting': 0.028646348317887208,
 'actively': 0.03170142019066156,
 'aly': 0.03170142019066156,
 'around': 0.03170142019066156,
 'azure': 0.03170142019066156,
 'basing': 0.03170142019066156,
 'besides': 0.03170142019066156,
 'bord': 0.03170142019066156,
 'british': 0.03170142019066156,
 'bucket': 0.03170142019066156,
 'buses': 0.03170142019066156,
 'captive': 0.03170142019066156,
 'cert': 0.03170142019066156,
 'configuration': 0.03170142019066156,
 'connect': 0.03170142019066156,
 'corona': 0.03170142019066156,
 'document': 0.03170142019066156,
 'drivers': 0.03170142019066156,
 'dubai': 0.03170142019066156,
 'elk': 0.031701420